In [2]:
import h5py
import numpy as np
import xarray as xr
import os

In [3]:
c_dir="F:/BT/NA"

In [4]:
for filename in os.listdir(c_dir):
    if 2018 >= int(filename[:4]):
        domain = os.path.abspath(c_dir)  
        dataFile = os.path.join(domain, filename) 
        data_dict = h5py.File(dataFile,"r")
        BT = data_dict["bt_rotate"]
        BT= np.squeeze(BT)
        lat = data_dict["lat_all"]
        lon = data_dict["lon_all"]
        time = data_dict["time_all"]
        wind = data_dict["wind_all"]
        area_200km=np.ones(np.size(BT,0))*np.nan
        area_300km=np.ones(np.size(BT,0))*np.nan
        bt_mean_200km=np.ones(np.size(BT,0))*np.nan
        bt_mean_300km=np.ones(np.size(BT,0))*np.nan
        for i in range(np.size(BT,0)):
            s=BT[i,:,:]
            y,x=np.ogrid[0:100,0:100]
            mask1 = (x-50)**2+(y-50)**2<=20**2 
            mask2 = (x-50)**2+(y-50)**2<=30**2
            s_1 = s[mask1]
            s_2 = s[mask2]
            area_200km[i]=np.size(s_1[s_1<=208])/np.size(s_1) *(20**2)*np.pi*100
            area_300km[i]=np.size(s_2[s_2<=208])/np.size(s_2) *(30**2)*np.pi*100
            bt_mean_200km[i]=np.nanmean(s_1)
            bt_mean_300km[i]=np.nanmean(s_2)
        
        area_200km[np.isnan(bt_mean_200km)]=np.nan
        area_300km[np.isnan(bt_mean_300km)]=np.nan
        
        BT_AREA = xr.Dataset({'area_200km':area_200km,'area_300km':area_300km,'bt_mean_200km':bt_mean_200km,'bt_mean_300km':bt_mean_300km},coords={'wind': np.squeeze(wind),'lat': np.squeeze(lat),
                                                                         'lon': np.squeeze(lon),'year': time[0],'month': time[1],'day': time[2],'hour': time[3]})
        BT_AREA.to_netcdf("F:/NA_BT_AREA/" +  dataFile[-15:-4] +".nc")

<ipython-input-4-d9a4816701ef>:25: RuntimeWarning: Mean of empty slice
  bt_mean_200km[i]=np.nanmean(s_1)
<ipython-input-4-d9a4816701ef>:26: RuntimeWarning: Mean of empty slice
  bt_mean_300km[i]=np.nanmean(s_2)
